# STATS401 Project 2


## Data Acquisition
---

### Query the transaction data of CryptoPunk from [Dune Analytics](https://dune.com/browse/queries/authored)


In [1]:
!pip install duneanalytics


In [2]:
from duneanalytics import DuneAnalytics
import time
import json
import pandas as pd
import numpy as np


In [3]:
def query_data_from_dune(data_mode):
    """
    Query data from Dune Analytics
    """
    # initialize client
    dune = DuneAnalytics('brucezzzzzzzyf@gmail.com', 'ZHANG0509yf')

    # try to login
    dune.login()

    # fetch token
    dune.fetch_auth_token()

    # set query ids
    if data_mode == 'punkBought':
        query_dict = {'2017': 673785,
                      '2018': 673784,
                      '2019': 673783,
                      '2020': 673781,
                      '2021': 673779,
                      '2022': 673771}
    elif data_mode == 'punkTransfer':
        query_dict = {'2017-2019': 674456,
                      '2020': 674472,
                      '2021': 674496,
                      '2022': 674500}

    # query data
    dune_data = {}
    for year in query_dict:
        query_id = query_dict[year]
        result_id = dune.query_result_id(query_id=query_id)
        dune_data[year] = dune.query_result(result_id)
        time.sleep(5)

    return dune_data


def save_data(dune_dict, file_predix, DUNE_DATA_PATH='./dune_data'):
    """
    Write data to json file
    """
    from datetime import date
    today = str(date.today())
    out_file = '{}/{}_{}.json'.format(DUNE_DATA_PATH, file_predix, today)
    with open(out_file, 'w') as f:
        json.dump(dune_dict, f)
    print('Data saved to {}'.format(out_file))


In [ ]:
dune_bt_dict = query_data_from_dune('punkBought')
save_data(dune_bt_dict, 'dune_bt_dict')


In [ ]:
dune_tf_dict = query_data_from_dune('punkTransfer')
save_data(dune_tf_dict, 'dune_tf_dict')


## Data Cleaning
---

In [10]:
import time
import json
import pandas as pd
import numpy as np

In [15]:
from datetime import date
today = str(date.today())

dune_bt_dict = json.load(
    open('./dune_data/dune_bt_dict_{}.json'.format(today)))
dune_tf_dict = json.load(
    open('./dune_data/dune_tf_dict_{}.json'.format(today)))


In [16]:
def index_trader(buyer, seller):
    trader = list(set(list(buyer.unique()) + list(seller.unique())))
    # print('Total {} traders'.format(len(trader)))

    return {trader[i]: i for i in range(len(trader))}


### 1. Transaction Database

- **tx_id**
- date_time
- from: trader_id
- yo: trader_id
- eth_price
- punk_id: punk_id


In [18]:
def create_tx_db(dune_bt_dict, dune_tf_dict):
    temp_idx = 0
    # punkBought data frame
    bt_data = dict()
    for year in dune_bt_dict:
        data_list = dune_bt_dict[year]['data']['get_result_by_result_id']
        bt_data.update({temp_idx+i: data_list[i]['data']
                       for i in range(len(data_list))})
        temp_idx += len(data_list)

    # punkTransfer data frame
    tf_data = dict()
    for year in dune_tf_dict:
        data_list = dune_tf_dict[year]['data']['get_result_by_result_id']
        tf_data.update({temp_idx+i: data_list[i]['data']
                       for i in range(len(data_list))})
        temp_idx += len(data_list)

    # concatenate data frames
    tx_db = pd.concat([pd.DataFrame.from_dict(bt_data, orient='index'),
                       pd.DataFrame.from_dict(tf_data, orient='index')])

    # set date_time
    tx_db['date_time'] = pd.to_datetime(tx_db['_date'] + ' ' + tx_db['_time'])

    # remove txs whose buyer is \x0000000000000000000000000000000000000000
    tx_db = tx_db[tx_db['buyer'] !=
                  '\\x0000000000000000000000000000000000000000']

    # reindex seller and buyer using trader_id
    trader_index_dict = index_trader(tx_db['buyer'], tx_db['seller'])
    tx_db['from'] = tx_db['seller'].apply(lambda x: trader_index_dict[x])
    tx_db['to'] = tx_db['buyer'].apply(lambda x: trader_index_dict[x])

    # sort by date_time
    tx_db.sort_values(by='date_time', inplace=True)

    # index transaction
    tx_db['tx_id'] = range(len(tx_db))
    tx_db = tx_db.loc[:, ['tx_id', 'date_time',
                          'from', 'to', 'eth_price', 'punk_id']]
    tx_db.set_index('tx_id', inplace=True)

    print('Total {} transactions'.format(len(tx_db)))
    print('Total {} unique traders'.format(len(trader_index_dict)))
    print('Total {} unique punk_id'.format(len(tx_db['punk_id'].unique())))

    return tx_db, trader_index_dict


tx_db, trader_index_dict = create_tx_db(dune_bt_dict, dune_tf_dict)
tx_db.to_csv('./database/tx_db.csv', index=True)
print('Transaction database saved to ./database/tx_db.csv')
tx_db


Total 40542 transactions
Total 8907 unique traders
Total 9007 unique punk_id
Transaction database saved to ./database/tx_db.csv


,date_time,from,to,eth_price,punk_id
tx_id,,,,,
0,2017-06-23 21:06:32,8291,7051,0.01,3134
1,2017-06-23 21:06:45,8291,3057,0.04,5719
2,2017-06-23 21:06:53,7051,7189,0.10,5056
3,2017-06-23 22:06:04,8291,2166,NaN,8138
4,2017-06-23 22:06:08,8291,3057,0.06,5624
...,...,...,...,...,...
40537,2022-05-06 03:05:13,1152,3314,NaN,4332
40538,2022-05-06 03:05:17,5755,4953,NaN,4516
40539,2022-05-06 03:05:17,4953,7748,NaN,4516


### 2. CryptoPunk

- **punk_id**
- img_url
- type (Alien, Ape, Zombie, Female, Male)
- skin_tone
- attr_count
- attributes
- current_owner: trader_id
- avg_price


In [19]:

def create_punk_db(punk_info_csv_list, CSV_PATH='./cp', PUNK_IMG_PATH='./punk_imgs'):
    punk_db = pd.read_csv('{}/{}'.format(CSV_PATH, punk_info_csv_list[0]))
    for i in range(1, len(punk_info_csv_list)):
        punk_df_temp = pd.read_csv(
            '{}/{}'.format(CSV_PATH, punk_info_csv_list[i]))
        punk_db = pd.concat([punk_db, punk_df_temp], axis=0)

    # rename columns
    punk_db.columns = ['punk_id', 'type', 'gender',
                       'skin_tone', 'attr_count', 'attributes']

    # strip type, gender, skin_tone
    punk_db['type'] = punk_db['type'].apply(lambda x: x.strip())
    punk_db['gender'] = punk_db['gender'].apply(lambda x: x.strip())
    punk_db['skin_tone'] = punk_db['skin_tone'].apply(lambda x: x.strip())
    
    # set cryptopunk image dir
    punk_db['img_dir'] = punk_db['punk_id'].apply(
        lambda x: '{}/{}.png'.format(PUNK_IMG_PATH, x))
    
    # make attributes as list
    punk_db['attributes'] = punk_db['attributes'].apply(lambda x: [i.strip() for i in x.split('/')])

    # set punk_id as index
    punk_db.set_index('punk_id', inplace=True)

    return punk_db


punk_info_csv_list = ['0-999.csv', '1000-1999.csv', '2000-2999.csv', '3000-3999.csv',
                      '4000-4999.csv', '5000-5999.csv', '6000-6999.csv',
                      '7000-7999.csv', '8000-8999.csv', '9000-9999.csv']

punk_db = create_punk_db(punk_info_csv_list)
punk_db.to_csv('./database/punk_db.csv', index=True)
print('Cryptopunk database saved to ./database/punk_db.csv')
punk_db


Cryptopunk database saved to ./database/punk_db.csv


,type,gender,skin_tone,attr_count,attributes,img_dir
punk_id,,,,,,
0,Human,Female,Medium,3,"[Green Eye Shadow, Earring, Blonde Bob]",./punk_imgs/0.png
1,Human,Male,Dark,2,"[Smile, Mohawk]",./punk_imgs/1.png
2,Human,Female,Light,1,[Wild Hair],./punk_imgs/2.png
3,Human,Male,Dark,3,"[Wild Hair, Nerd Glasses, Pipe]",./punk_imgs/3.png
4,Human,Male,Medium,4,"[Big Shades, Wild Hair, Earring, Goat]",./punk_imgs/4.png
...,...,...,...,...,...,...
9995,Human,Female,Albino,2,"[Purple Eye Shadow, Straight Hair Dark]",./punk_imgs/9995.png
9996,Human,Male,Light,4,"[Cigarette, Earring, Crazy Hair, Smile]",./punk_imgs/9996.png
9997,Zombie,Male,,2,"[Front Beard, Cap Forward]",./punk_imgs/9997.png


### 3. Trader

- **trader_id**
- address
- tx_involved_count
- tx_involved: list of tx_id
- frequent_skin_tone
- frequent_gender


In [20]:

def find_most_frequent_skin_tone(tx_involved_list, tx_db):
    skin_tones = list()
    for tx_id in tx_involved_list:
        punk_id = tx_db.loc[tx_id, 'punk_id']
        skin_tones.append(punk_db.loc[punk_id, 'skin_tone'])
    return max(set(skin_tones), key=skin_tones.count)


def find_most_frequent_gender(tx_involved_list, tx_db):
    genders = list()
    for tx_id in tx_involved_list:
        punk_id = tx_db.loc[tx_id, 'punk_id']
        genders.append(punk_db.loc[punk_id, 'gender'])
    return max(set(genders), key=genders.count)


def create_trader_db(tx_db, trader_index_dict):
    trader_db = pd.DataFrame.from_dict(
        trader_index_dict, orient='index', columns=['trader_id'])

    # set address for each trader
    trader_db.reset_index(inplace=True)
    trader_db.rename(columns={'index': 'address'}, inplace=True)

    # set trader_id as index
    trader_db.sort_values(by='trader_id', inplace=True)

    # get the involved tx
    trader_db['tx_involved_count'] = trader_db['trader_id'].apply(
        lambda x: len(tx_db[tx_db['from'] == x]) + len(tx_db[tx_db['to'] == x]))
    trader_db['tx_involved'] = trader_db['trader_id'].apply(lambda x: sorted(list(tx_db[tx_db['from'] == x].index) +
                                                            list(tx_db[tx_db['to'] == x].index)))

    # get the skin tone that each trader most frequently trades with
    trader_db['frequent_skin_tone'] = trader_db['tx_involved'].apply(
        lambda x: find_most_frequent_skin_tone(x, tx_db))

    # get the gender that each trader most frequently trades with
    trader_db['frequent_gender'] = trader_db['tx_involved'].apply(
        lambda x: find_most_frequent_gender(x, tx_db))

    # set trader_id as index
    trader_db.set_index('trader_id', inplace=True)

    return trader_db


trader_db = create_trader_db(tx_db, trader_index_dict)
trader_db.to_csv('./database/trader_db.csv', index=True)
print('Trader database saved to ./database/trader_db.csv')
trader_db


Trader database saved to ./database/trader_db.csv


,address,tx_involved_count,tx_involved,frequent_skin_tone,frequent_gender
trader_id,,,,,
0,\x94eba34e650787b6dbf89f9dd0d31923521dc9e3,1,[39161],Dark,Female
1,\x90c476355205618fe342abfc4555226cd8e41879,2,"[18220, 28149]",Dark,Male
2,\x1cbb87bb9a4dd0316189eede2277a58590dc124a,14,"[27623, 27720, 27812, 27813, 27814, 27831, 290...",,Female
3,\x537dc8c046259fab181611035a529ea496859e6d,2,"[24258, 30946]",Light,Male
4,\x3a6cb57e87ba21f48304ff4a0f6ebf81d262a22c,2,"[18409, 29963]",Light,Male
...,...,...,...,...,...
8902,\x0a8fa7821e73e6bac4e22e04575424e8bd931f0f,4,"[18976, 18977, 19031, 19032]",Light,Male
8903,\x8c2b143b0276bafd2613bb41d7058583cf6706c7,1,[37421],Dark,Male
8904,\xda3c325ab45b30aeb476b026fe6a777443ca04f3,2,"[35294, 35295]",Light,Female


## Data Engineering for Visualization
---

Visualization 1: `vis1.ipynb`

Visualization 2: `vis2.ipynb`

Visualization 3: `vis3.ipynb`

Visualization 4: `vis4.ipynb`

Visualization 5: `vis5.ipynb`